In [1]:
from utils.predictions_content import *
from utils.predictions_notes import *
from utils.collaborative import *
from collections import OrderedDict

/Users/constancescherer/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#path = "/Vrac/PLDAC_addic7ed/ratings/ratings_imdb/users"
path_ratings = "/Users/constancescherer/Desktop/ratings/ratings_imdb/users"
path_d_user = "/Users/constancescherer/Desktop/pickles/d_user.p"
#d_user= get_d_user(path)
with open(path_d_user, 'rb') as pickle_file:
    d_user = pickle.load(pickle_file)

liste_series = get_liste_series(d_user)
data = get_data(d_user)
all_data, num_user, num_item = get_all_data(data)
train, train_mat, test = get_train_test(num_user, num_item, all_data, test_size=20)
mean, u_means, i_means,U_ksvd, I_ksvd =  get_Uksvd_Iksvd(train, train_mat, num_user, num_item)
d_username_id, d_itemname_id, Full = create_sparse_mat(data)

path_series = "/Users/constancescherer/Desktop/addic7ed_final"
#path_series = '/Vrac/PLDAC_addic7ed/addic7ed_clean'
d_info, d_name = getDicts(path_series)
d_ind = reverse_dict(d_name)
d_titre_filename = get_d_titre_filename("titles/title-filename.txt")
d_filename_titre = reverse_dict(d_titre_filename)
d_id_username = reverse_dict(d_username_id)
d_id_serie = reverse_dict(d_itemname_id)

In [3]:
d_serie = dict()
for username, d_s in d_user.items() :
    for serie, rating in d_s.items() :
        if serie in d_serie.keys() :
            d_serie[serie] += 1
        else :
            d_serie[serie] = 1

In [5]:
d_serie['aidan-5']

1

In [4]:
d_user['poe-48833']

{'blade-of-the-immortal': 10.0,
 'detectorists': 10.0,
 'the-civil-war': 10.0,
 'the-walking-dead': 9.0,
 'the-x-files': 10.0}

In [5]:
d_user['ADtheThird']

{'banshee': 9.0,
 'castle': 10.0,
 'common': 8.0,
 'common-law': 8.0,
 'continuum': 7.0,
 'psych': 10.0,
 'the-vampire-diaries': 9.0}

In [6]:
username = 'ADtheThird'

In [7]:
uid = d_username_id[username]
d_notes = dict()
for serie, iid in d_itemname_id.items() :
    if serie not in d_user[username].keys() :
        # prediction
        p = pred_func_ksvd(uid, iid, U_ksvd, I_ksvd, u_means, i_means, mean)
        if p > 10 :
            p = 10
        d_notes[serie] = p 

In [8]:
d_notes

{'boardwalk-empire': 9.922663800375801,
 'game-of-thrones': 9.341958528594041,
 'episodes': 9.004329953874858,
 'american-horror-story': 8.017820267211224,
 'shameless': 9.207699345237941,
 'being-human': 7.084976801796564,
 'ray-donovan': 9.044885226658412,
 'masters-of-sex': 7.810643185350231,
 'the-affair': 7.586316640036446,
 'empire': 9.811399462315942,
 'last-week-tonight-with-john-oliver': 10,
 'the-good-wife': 9.499750252056838,
 'the-walking-dead': 8.136784546833997,
 'terra-nova': 7.401110634851297,
 'the-shield': 10,
 'homeland': 8.640592219746711,
 'lost': 8.653831310697182,
 'breaking-bad': 10,
 'nova': 7.453645557710975,
 'rectify': 9.153507448871682,
 'under-the-dome': 6.239651543753487,
 'true-detective': 9.094773360661097,
 'two-and-a-half-men': 7.147281877888449,
 'cambridge-spies': 9.409294469015938,
 'empire-falls': 8.191479399964049,
 'profiler': 7.408058507101876,
 'angels-in-america': 8.9171177646092,
 'csi-miami': 6.777888108813572,
 'the-west-wing': 9.825713329

In [9]:
sorted_x = sorted(d_notes.items(), key=lambda kv: kv[1])
sorted_x.reverse()

sorted_dict = OrderedDict(sorted_x)

In [10]:
sorted_x[0:10]

[('smith', 10),
 ('blott-on-the-landscape', 10),
 ('off-limits', 10),
 ('better-man', 10),
 ('con-man', 10),
 ('inside-the-human-body', 10),
 ('kungamordet', 10),
 ('guys-with-kids', 10),
 ('up-pompeii-', 10),
 ('z-rock', 10)]

In [17]:
keys_l = list(sorted_dict.keys())

In [18]:
keys_l[:3]

['smith', 'blott-on-the-landscape', 'off-limits']

In [30]:
## Getting the truth values
truth_tr = np.array([rating for (uid,iid),rating in train_mat.items()])
truth_te = np.array([rating for uid,iid,rating in test])

prediction_tr = np.array([pred_func_ksvd(u,i, U_ksvd, I_ksvd, u_means, i_means, mean) for (u,i),rating in train_mat.items()])
prediction_te = np.array([pred_func_ksvd(u,i, U_ksvd, I_ksvd, u_means, i_means, mean) for u,i,rating in test])


print("Training Error:")
print("MSE:",  MSE_err(prediction_tr,truth_tr))
print("MAE:",  MAE_err(prediction_tr,truth_tr))
    
print("Test Error:")
print("MSE:",  MSE_err(prediction_te,truth_te))
print("MAE:",  MAE_err(prediction_te,truth_te))

Training Error:
MSE: 1.5642711576695214
MAE: 0.799072435618619
Test Error:
MSE: 5.645687708445735
MAE: 1.8053423021007617


In [13]:
prediction_tr, prediction_te = predictions_NMF(train_mat,test, 100, num_user, num_item)
print("Training Error:")
print("MSE:",  MSE_err(prediction_tr,truth_tr))
print("MAE:",  MAE_err(prediction_tr,truth_tr))
    
print("Test Error:")
print("MSE:",  MSE_err(prediction_te,truth_te))
print("MAE:",  MAE_err(prediction_te,truth_te))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Training Error:
MSE: 0.0055299895
MAE: 0.0055299895
Test Error:
MSE: 9.278471711976488
MAE: 2.358559882439383
